## 3. Search Data

In [1]:
import os
import json
import requests

from rasterio.features import bounds as featureBounds

from ipyleaflet import Map, basemaps, TileLayer, basemap_to_tiles, GeoJSON

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Endpoint variables
titiler_endpoint = "{ENDPOINT}"
stac_endpoint = "https://earth-search.aws.element84.com/v0/search"

# Make sure both are up
assert requests.get(f"{titiler_endpoint}/docs").status_code == 200
assert requests.get(stac_endpoint).status_code == 200

More info: https://github.com/radiantearth/stac-api-spec for more documentation about the stac API

1. AOI

You can use geojson.io to define your search AOI

In [3]:
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -2.83447265625,
              4.12728532324537
            ],
            [
              2.120361328125,
              4.12728532324537
            ],
            [
              2.120361328125,
              8.254982704877875
            ],
            [
              -2.83447265625,
              8.254982704877875
            ],
            [
              -2.83447265625,
              4.12728532324537
            ]
          ]
        ]
      }
    }
  ]
}

bounds = featureBounds(geojson)

m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=6
)

geo_json = GeoJSON(data=geojson)
m.add_layer(geo_json)
m

Map(center=[6.191134014061623, -0.3570556640625], controls=(ZoomControl(options=[&#39;position&#39;, &#39;zoom_in_text&#39;, &#39;…

2. Define dates and other filters

In [6]:

# POST body
query = {
    "collections": ["landsat-8-l1-c1"],
    "query": {
        "eo:cloud_cover": {
            "lt": 3
        },
    },
    "limit": 1000,
    "fields": {
      'include': ['id', 'properties.datetime', 'properties.eo:cloud_cover'],  # This will limit the size of returned body
      'exclude': ['assets', 'links']  # This will limit the size of returned body
    },
    "sortby": [
        {
            "field": "properties.eo:cloud_cover",
            "direction": "desc"
        },
    ]
}

# POST Headers
headers = {
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip",
    "Accept": "application/geo+json",
}

data = requests.post(stac_endpoint, headers=headers, json=query).json()
print("Results context:")
print(data["context"])

sceneid = [f["id"] for f in data["features"]]
cloudcover = [f["properties"]["eo:cloud_cover"] for f in data["features"]]
dates = [f["properties"]["datetime"][0:10] for f in data["features"]]

Results context:
{&#39;page&#39;: 1, &#39;limit&#39;: 1000, &#39;matched&#39;: 1676, &#39;returned&#39;: 1000}


In [7]:
data["features"][0]

{&#39;bbox&#39;: [17.52159, 37.79375, 20.29877, 39.99183],
 &#39;geometry&#39;: {&#39;coordinates&#39;: [[[18.11632174572913, 39.99120072125492],
    [20.295303143374994, 39.543111151569946],
    [19.70135355249007, 37.79500906248807],
    [17.524595993314055, 38.249780621426254],
    [18.11632174572913, 39.99120072125492]]],
  &#39;type&#39;: &#39;Polygon&#39;},
 &#39;id&#39;: &#39;LC08_L1TP_186033_20200913_20200920_01_T1&#39;,
 &#39;collection&#39;: &#39;landsat-8-l1-c1&#39;,
 &#39;type&#39;: &#39;Feature&#39;,
 &#39;properties&#39;: {&#39;datetime&#39;: &#39;2020-09-13T09:23:30.572134Z&#39;,
  &#39;eo:cloud_cover&#39;: 2.98}}

In [8]:
bounds = data["features"][0]["bbox"]

m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=8
)

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
m.add_layer(geo_json)
m

Map(center=[38.892790000000005, 18.91018], controls=(ZoomControl(options=[&#39;position&#39;, &#39;zoom_in_text&#39;, &#39;zoom_in…

In [9]:
# Print what band are available
from rio_tiler_pds.landsat.aws.landsat8 import landsat8_valid_bands
print(landsat8_valid_bands)

(&#39;B1&#39;, &#39;B2&#39;, &#39;B3&#39;, &#39;B4&#39;, &#39;B5&#39;, &#39;B6&#39;, &#39;B7&#39;, &#39;B8&#39;, &#39;B9&#39;, &#39;B10&#39;, &#39;B11&#39;, &#39;BQA&#39;)


In [14]:
# Fetch TileJSON
# For this example we use the first `sceneid` return from the STAC API
# and we sent the Bands to B04,B03,B02 which are red,green,blue
print(sceneid[4])
data = requests.get(f"{titiler_endpoint}/scenes/landsat/tilejson.json?sceneid={sceneid[4]}&bands=B4,B3,B2&rescale=0,5000").json()
print(data)

LC08_L1TP_146032_20200905_20200917_01_T1
{&#39;tilejson&#39;: &#39;2.2.0&#39;, &#39;name&#39;: &#39;LC08_L1TP_146032_20200905_20200917_01_T1&#39;, &#39;version&#39;: &#39;1.0.0&#39;, &#39;scheme&#39;: &#39;xyz&#39;, &#39;tiles&#39;: [&#39;https://c50qa6bhpe.execute-api.us-west-2.amazonaws.com/landsat/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?sceneid=LC08_L1TP_146032_20200905_20200917_01_T1&amp;bands=B4%2CB3%2CB2&amp;rescale=0%2C5000&#39;], &#39;minzoom&#39;: 7, &#39;maxzoom&#39;: 12, &#39;bounds&#39;: [79.7992, 39.25692, 82.57623, 41.38507], &#39;center&#39;: [81.187715, 40.320994999999996, 7]}


In [15]:
bounds = data["bounds"]
m = Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=10
)

tiles = TileLayer(
    url=data["tiles"][0],
    min_zoom=data["minzoom"],
    max_zoom=data["maxzoom"],
    opacity=1
)
m.add_layer(tiles)
m

Map(center=[40.320994999999996, 81.187715], controls=(ZoomControl(options=[&#39;position&#39;, &#39;zoom_in_text&#39;, &#39;zoom_i…